# HLY 1801 Catch

- we didn't use these for the soundspeed/absorption, we used 2017 (add to methods)
- what would the impact be on backscatter, because ranges are short and frequency is low,  if the ss varied by 10% backscatter averaged would

to answer review
- yes i agree, i used water column averages from a previous year.
for text:
I used values for 2017 because this is reasonable because it is relatively representative, within the context of 4 years
- here's rationally how different it would be, and what would that mean for my abundance 

1. Age-0 gadids are largely retained change the title
2. Find every place in the paper that says arctic cod and smooth it down. 
2. 40% unidentified gadids, genetic testing is being done

In [3]:
import pandas as pd

In [5]:
hly2018 = pd.read_csv('../../data/catchData/MethotCatch.csv')
gAcod = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae') & (hly2018.Species=='Arctic Cod')])
gScod = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae') & (hly2018.Species=='Saffron Cod')])
gAll = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae')])
print('Total Gadids north of 69N: ',gAll)
print('Total Arctic cod north of 69N: ',gAcod,',',(gAcod/gAll)*100,'%')
print('Total Saffron cod north of 69N: ',gScod,',',(gScod/gAll)*100,'%')

Total Gadids north of 69N:  116
Total Arctic cod north of 69N:  82 , 70.6896551724138 %
Total Saffron cod north of 69N:  27 , 23.275862068965516 %


# 2017 and 2019 CPUE
Number of fish per 100,000 cubic meters

In [6]:
eventStats = pd.read_csv('../../data/catchData/AIESMarinovichEventData.csv')
gadidRatios = pd.read_csv('../../data/catchData/eventRatios.csv')
# fix 1 missing haul which has no genetics and all pollock
gadidRatios = gadidRatios.append({'SURVEY':201901, 'EVENT_ID':404, 'acod':0, 'pollock':0, 'acodRatio':0,'pollockRatio':1},ignore_index=True)
gadidRatios= gadidRatios.drop(columns=['Unnamed: 0'])
eventStats['cpueF'] = 1/(eventStats.VOL_FILTERED/100000)
eventStats=eventStats[['SURVEY','EVENT_ID','EQ_LONGITUDE', 'EQ_LATITUDE','VOL_FILTERED','cpueF']]
os2017=pd.read_csv('../../data/catchData/OS201701MidwaterCatchSummary.csv')
os2019=pd.read_csv('../../data/catchData/OS201901MidwaterCatchSummary.csv')
catch = pd.concat([os2017,os2019])
catch = catch.rename(columns={'CLAMS_EVENT_NUMBER':'EVENT_ID'})
catchMerged = pd.merge(catch, eventStats, on=['SURVEY','EVENT_ID'], how='left')
catchMerged = catchMerged[catchMerged.SPECIES_CODE<40000]
catchMerged['cpueCatch'] = catchMerged.TOTAL_NUMBER_IN_HAUL*catchMerged.cpueF
gadids = catchMerged[((catchMerged.SPECIES_CODE== 21725)|(catchMerged.SPECIES_CODE== 21740)|(catchMerged.SPECIES_CODE== 21744))][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE', 'EQ_LATITUDE','SPECIES_CODE','cpueCatch']]
nongadids = catchMerged[((catchMerged.SPECIES_CODE!= 21725)&(catchMerged.SPECIES_CODE!= 21740)&(catchMerged.SPECIES_CODE!= 21744))][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE', 'EQ_LATITUDE','SPECIES_CODE','cpueCatch']]

dsList = []
for df in [gadids[gadids.SURVEY==201701],gadids[gadids.SURVEY==201901]]:
    for event in df.EVENT_ID.unique():
        for spec in [21725, 21740]:
            ds = df[(df.EVENT_ID == event)].iloc[0][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE','EQ_LATITUDE']]
            tGadids = df[(df.EVENT_ID==event)&((df.SPECIES_CODE== 21725)|(df.SPECIES_CODE== 21740)|(df.SPECIES_CODE== 21744))].cpueCatch.sum()
            if spec == 21725:
                ds['SPECIES_CODE']= 21725
                ds['cpueCatch']=tGadids*gadidRatios[(gadidRatios.SURVEY==df.SURVEY.unique()[0]) & (gadidRatios.EVENT_ID==event)].acodRatio.values[0]
            elif spec == 21740:
                ds['SPECIES_CODE']= 21740
                ds['cpueCatch']=tGadids*gadidRatios[(gadidRatios.SURVEY==df.SURVEY.unique()[0]) & (gadidRatios.EVENT_ID==event)].pollockRatio.values[0]
            dsList.append(ds)
newGadids = pd.DataFrame(dsList)
newCatch = nongadids.append(newGadids,sort=False)

In [7]:
dsList = []
for df in [newCatch[newCatch.SURVEY==201701],newCatch[newCatch.SURVEY==201901]]:
    for event in df.EVENT_ID.unique(): 
        if df[(df.EVENT_ID == event) & (df.SPECIES_CODE==21725)].empty:
            ds = df[(df.EVENT_ID == event)].iloc[0][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE','EQ_LATITUDE']]
            ds['propAcod']=0
        else:
            ds = df[(df.EVENT_ID == event)].iloc[0][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE','EQ_LATITUDE']]
            ds['propAcod']=df[(df.EVENT_ID == event) & (df.SPECIES_CODE==21725)].cpueCatch.values[0]/df[df.EVENT_ID == event].cpueCatch.sum()
        dsList.append(ds)
propAcod = pd.DataFrame(dsList)

In [8]:
(newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),\
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum())

(0.8546649835281178, 0.34180719489864725)

In [9]:
newCatchNoSL = newCatch[(newCatch.SPECIES_CODE != 20203) & (newCatch.SPECIES_CODE != 20204)]
(newCatchNoSL[(newCatchNoSL.SURVEY==201701)& (newCatchNoSL.EQ_LATITUDE >69)& (newCatchNoSL.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatchNoSL[(newCatchNoSL.SURVEY==201701)& (newCatchNoSL.EQ_LATITUDE >69)].cpueCatch.sum(),\
newCatchNoSL[(newCatchNoSL.SURVEY==201901)& (newCatchNoSL.EQ_LATITUDE >69)& (newCatchNoSL.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatchNoSL[(newCatchNoSL.SURVEY==201901)& (newCatchNoSL.EQ_LATITUDE >69)].cpueCatch.sum())

(0.8548619686636874, 0.4554744663143731)